In [ ]:
!pip install scikit-learn==0.24.1  aif360==0.3.0  tensorflow==2.4.0 nodejs==0.1.1 ipywidgets==7.6.3 lime==0.2.0.1 #aix360==0.2.1 

In [ ]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = keras.models.load_model('model')

In [ ]:
import pandas as pd
df = pd.read_csv('fairface_label_val.csv')

In [ ]:
from PIL import Image
from numpy import asarray

dataset = []

for i, row in df.iterrows():
    image = 'data/val/'+row['gender'].lower()+'/'+row['file'].split('/')[1] 
    image = Image.open(image)
    image = image.resize((244,244), Image.ANTIALIAS)
    data = asarray(image)
    data.shape = (1, 244, 244, 3)
    label = 0 if row['gender']=="Female" else 1
    prediction = model.predict(data)
    prediction = tf.math.argmax(prediction,1).numpy()[0] #de-onehotencode
    dataset.append([label,prediction])

In [ ]:
import numpy as np
dataset = np.array(dataset)

In [ ]:
import pandas as pd
df = pd.read_csv('fairface_label_val.csv')

In [ ]:
df['labels'] = dataset[:,0].tolist()
df['prediction'] = dataset[:,1].tolist()

In [ ]:
df["age_index"] = df.age.apply(lambda x: list(df.age.unique()).index(x))
df["gender_index"] = df.gender.apply(lambda x: list(df.gender.unique()).index(x))
df["race_index"] = df.race.apply(lambda x: list(df.race.unique()).index(x))
df["missclassified"] = df.apply(lambda d: 1 if d.labels!=d.prediction else 0, axis=1)

In [ ]:
from aif360.datasets import BinaryLabelDataset

unprivileged_groups = [{'race_index': 2}]
privileged_groups = [{'race_index': 1 }]
                     
                   
favorable_label = 0
unfavorable_label = 1

df_for_aif360 = df[["age_index", "gender_index", "race_index", "missclassified"]]



df_for_aif360

In [ ]:

aif360_dataset = BinaryLabelDataset(
    favorable_label=favorable_label,
    unfavorable_label=unfavorable_label,
    df=df_for_aif360,
    label_names=['missclassified'],
    protected_attribute_names=['race_index'],
    unprivileged_protected_attributes=unprivileged_groups)

In [ ]:
from aif360.metrics import BinaryLabelDatasetMetric

metric_orig_train = BinaryLabelDatasetMetric(aif360_dataset, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

In [ ]:
metric_orig_train.statistical_parity_difference()